### Checkout my [Twitter(@rohanpaul_ai)](https://twitter.com/rohanpaul_ai) for daily LLM bits

##  Fine Tuning Phi 1_5 with PEFT and QLoRA

# [Link to my Youtube Video Explaining this whole Notebook](https://www.youtube.com/watch?v=J0RbOtLrJhQ&list=PLxqBkZuBynVTzqUQCQFgetR97y1X_1uCI&index=10&ab_channel=Rohan-Paul-AI)

[![Imgur](https://imgur.com/ensYOlt.png)](https://www.youtube.com/watch?v=J0RbOtLrJhQ&list=PLxqBkZuBynVTzqUQCQFgetR97y1X_1uCI&index=10&ab_channel=Rohan-Paul-AI)



In [1]:
!pip install accelerate transformers einops datasets peft bitsandbytes --upgrade

In [2]:
# from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import os

In [4]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-1_5",
    device_map={"":0},
    trust_remote_code=True,
    quantization_config=bnb_config
)

In [6]:
model

## A little hack to make `get_peft_model()` method faster by avoiding `kaiming_uniform` initialization

In [7]:
from bitsandbytes.nn.modules import Linear8bitLt, Linear4bit
import torch.nn.init as init  # Importing the init module from PyTorch
import bitsandbytes as bnb
from contextlib import contextmanager

def noop (x=None, *args, **kwargs):
    "Do nothing"
    return x

@contextmanager
def no_kaiming():
    old_iku = init.kaiming_uniform_
    init.kaiming_uniform_ = noop
    try: yield
    finally: init.kaiming_uniform_ = old_iku

_old_8init = Linear8bitLt.__init__
_old_4init = Linear4bit.__init__

def _new_4init(self, input_features, output_features, bias=True,
               device=None, **kwargs):
    with no_kaiming():
        return _old_4init(self, input_features, output_features, bias=bias,
                          device=device, **kwargs)



def _new_8init(self, input_features, output_features, bias=True, has_fp16_weights=True,
              memory_efficient_backward=False, threshold=0.0, index=None, device=None):
    with no_kaiming():
        return _old_8init(self, input_features, output_features, bias=bias, has_fp16_weights=has_fp16_weights,
                          memory_efficient_backward=memory_efficient_backward, threshold=threshold, index=index, device=device)

Linear8bitLt.__init__ = _new_8init
Linear4bit.__init__ = _new_4init

In [8]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["Wqkv", "out_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 4,718,592 || all params: 1,422,989,312 || trainable%: 0.3315971497613047


![](assets/lora-4.png)

In [9]:
# print(model)

In [10]:
def tokenize(sample):
    tokenized_text =  tokenizer(sample["text"], padding=True, truncation=True, max_length=512)
    return tokenized_text

In [12]:
data = load_dataset("BI55/MedText", "main", split="train")

data_df = data.to_pandas()

In [13]:
data_df["text"] = data_df[["Prompt", "Completion"]].apply(lambda x: "Prompt: " + x["Prompt"] + " Completion: " + x["Completion"], axis=1)

In [ ]:
data = Dataset.from_pandas(data_df)

tokenized_data = data.map(tokenize, batched=True, desc="Tokenizing data", remove_columns=data.column_names)

tokenized_data

In [15]:
training_arguments = TrainingArguments(
        output_dir="phi-1_5-finetuned-med-text",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        max_steps=1000,
        num_train_epochs=1
    )

In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_data,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

# trainer.push_to_hub()

## Saving

In [17]:
model.save_pretrained("phi-1_5-finetuned-med-text")

In [18]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", trust_remote_code=True, torch_dtype=torch.float32)

peft_model = PeftModel.from_pretrained(model, "phi-1_5-finetuned-med-text", from_transformers=True)

model = peft_model.merge_and_unload()

# model

In [19]:
model.save_pretrained("phi-1_5-finetuned-med-text")

# model.push_to_hub("llm-exp/phi-1_5-finetuned-med-text")

## The overall flow

1. load the base model

2. train the base model

3. save the LoRA adapter

4. reload the base model at full precision

5. merge the LoRA weights with the base model

6. save


# Inference

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("phi-1_5-finetuned-med-text", trust_remote_code=True, torch_dtype=torch.float32)

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

inputs = tokenizer('I am having a headache with but no fever - what could be the cause', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=512)

text = tokenizer.batch_decode(outputs)[0]

print(text)
